# Text Summarization [BART Fine Tune]

## 01. Install and Import Libraries

In [2]:
!pip install -U datasets transformers[torch] rouge-score evaluate accelerate nltk
!pip install comet-ml

In [3]:
import comet_ml
import os 

# Set environment variables for distributed training
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Use GPUs 0 and 1
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12355"
comet_ml.login(project_name="Summarizer")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


COMET INFO: Valid Comet API Key saved in /teamspace/studios/this_studio/.comet.config (set COMET_CONFIG to change where it is saved).


In [4]:
from huggingface_hub import notebook_login

# Paste hugging face token with write permission enabled and log in
notebook_login()

In [5]:
import datasets
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("billsum")
metric = load("rouge")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
max_input_length = 1024
max_target_length = 128

prefix = "summarize: "

# Preprocess input data
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_billsum = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Dynamic Padding batch wise
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=model_checkpoint,
                                       label_pad_token_id=tokenizer.pad_token_id)

# Load Pre-Trained Bart-Large Checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [14]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]

# Define hyperparams
args = Seq2SeqTrainingArguments(
    f"bart-large-finetuned-billsum",
    num_train_epochs=3,                       # No of epochs to train
    per_device_train_batch_size=batch_size,   # Batch size for training per device
    per_device_eval_batch_size=batch_size,    # Batch size for evaluation per device
    learning_rate=2e-5,                       # Learning rate for optimizer
    warmup_steps=500,                         # No of warmup steps for the learning rate scheduler
    weight_decay=0.01,                        # Weight decay coefficient for regularization
    logging_dir='./logs',                     # Directory for logging training information
    load_best_model_at_end=True,              # Whether to load the best model from checkpoints at the end of training
    logging_steps=1000,                       # Log training metrics every `logging_steps` steps
    save_steps=1000,                          # Save model checkpoints every `save_steps` steps
    save_total_limit=2,                       # Save no of checkpoints
    eval_strategy="steps",                    # When to run evaluation during training: steps, epochs or none
    predict_with_generate=True,               # Generate preds during eval for metrics score
    generation_max_length=32,                 # Generation length | NOTE: Evaluation becomes faster if choosen small gen_length
    fp16=True,                                # Floating point 16 precision
    report_to=["comet_ml"],                   # Experiment Tracker: CometML
    push_to_hub=True,                         # Save checkpoint in Hugging Face Hub
)

In [15]:
import nltk
import numpy as np
import evaluate

rouge = evaluate.load("rouge")
nltk.download('punkt')
nltk.download('punkt_tab')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # RougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Replace -100 in labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"].select(range(3000)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [17]:
from accelerate import Accelerator

# Initialize Accelerator and Trainer
Accelerator()
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1000,1.527900,1.363842,34.285300,26.162700,31.897000,32.639900,31.999000
2000,1.323700,1.235749,34.705500,26.793600,32.381100,33.082300,31.997333
3000,1.159400,1.224593,34.697500,27.096400,32.532600,33.188300,31.982000
4000,1.102900,1.209232,34.496900,26.910700,32.364400,33.048100,31.998667
5000,1.046100,1.176941,35.241900,27.603800,33.033900,33.684900,31.990333
6000,0.953500,1.195765,34.713800,27.218500,32.557300,33.204300,31.994667
7000,0.937300,1.159985,35.174100,27.619900,32.961800,33.618100,31.978333
8000,0.850600,1.193973,34.897600,27.445500,32.758100,33.401300,31.990000
9000,0.834100,1.171598,35.119100,27.685600,32.982200,33.622100,31.985333
10000,0.808300,1.191641,35.183900,27.701300,32.995000,33.613100,31.988000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a G

TrainOutput(global_step=11845, training_loss=1.009852511855609, metrics={'train_runtime': 17978.2056, 'train_samples_per_second': 5.27, 'train_steps_per_second': 0.659, 'total_flos': 2.0532232552710144e+17, 'train_loss': 1.009852511855609, 'epoch': 5.0})

In [19]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/luluw/bart-large-finetuned-billsum/commit/1d8292e426b90bd2c3cf70dfbf58f05be19ee4bc', commit_message='End of training', commit_description='', oid='1d8292e426b90bd2c3cf70dfbf58f05be19ee4bc', pr_url=None, pr_revision=None, pr_num=None)